<a href="https://colab.research.google.com/github/aldrich1221/recommendation-system/blob/main/GNN/GraphSAGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

def sampling(sourceNodes,samplingNum,nodeTable):
  results=[]
  for sid in sourceNodes:
    sampleNeighbor=np.random.choice(nodeTable[sid],size=(samplingNum,))
    results.append(sampleNeighbor)
  return np.asarray(results).flatten()

def multiSampling(sourceNodes,samplingNum,nodeTable):
  multiSamplingResult=[sourceNodes]
  for i,samplenum in enumerate(samplingNum):
    hopKResult=sampling(multiSamplingResult[i],samplenum,nodeTable)
    multiSamplingResult.append(hopKResult)
  return multiSamplingResult



In [ ]:
class Aggregator(nn.Module):
  def __init__(self,inputDim,outputDim,aggMethod='mean'):
    super(Aggregator,self).__init__()
    self.inputDim=inputDim
    self.outputDim=outputDim
    self.useBias=useBias
    self.aggMethod=aggMethod
    self.weight=nn.Parameter(torch.Tensor(inputDim,outputDim))
   
    self.bias=nn.Parameter(torch.Tensor(outputDim))
    self.reset()

  def reset(self):
    init.kaiming_uniform_(self.weight)
    if self.useBias:
      init.zeros_(self.bias)
  def forward(self,neighborFeature):
    if self.aggMethod=='mean':
      aggNeighbor=neighborFeature.mean(dim=1)
    elif self.aggMethod=='max':
      aggNeighbor=neighborFeature.max(dim=1)
    else:
      raise ValueError("please  ues mean aggregation")
    neighborHidden=torch.matmul(aggNeighbor,self.weight)+self.bias
   
    return neighborHidden


